# YOLOv3 커스텀 데이터 학습

copy from https://articlearn.id/article/e5571bb0-how-to-train-yolov3-on-google-colab-to-de/



# 레이블링 데이터 위치

구글 드라이브 tmp/ 폴더에 laptops_labeled.zip 데이터 파일을 준비한다.

<br>
laptops_labeled.zip 파일 내용

```
1.jpg
1.txt
2.jpg
2.txt
...
```

<br>
레이블 데이터 내용

1.txt
```
0 0.498403 0.541966 0.559105 0.585132
```
한줄만 있다. 

첫번째 0은 클래스 인덱스, 

0.498403, 0.541966은 박스의 중심

0.559105, 0.585132는 박스의 폭과 높이





# 탐지할 대상 이름 설정

In [1]:
CUSTOM_OBJECT_NAMES = ['laptop', 'keyboard']

# 0. 임포트

In [2]:
import cv2
import matplotlib.pyplot as plt
import glob


# 1. 구글 드라이브 마운트, mydrive로 링크

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ln -s /content/gdrive/MyDrive/ /mydrive

# 2. Darknet 다운받아서 컴파일

## 프로젝트 다운로드

In [5]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15386, done.
remote: Total 15386 (delta 0), reused 0 (delta 0), pack-reused 15386
Receiving objects: 100% (15386/15386), 14.01 MiB | 2.57 MiB/s, done.
Resolving deltas: 100% (10346/10346), done.


OpenCV, GPU, CuDNN 사용 관련 설정하고 컴파일

In [6]:
%cd darknet

/content/darknet


## 컴파일 설정 파일 수정

In [7]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

## 컴파일

In [8]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

# 3. 데이터 준비
- darknet/data/obj에 latops_labeled.zip 압축 풀기
- classes.names와 training.data 파일 생성
- training.txt 파일 생성


In [11]:
!ls -al /mydrive/tmp/laptops_labeled.zip

-rw------- 1 root root 698575 Jan  5 02:59 /mydrive/tmp/laptops_labeled.zip


## data/obj/laptops_labeled 폴더에 압축 풀기

In [12]:
!rm -rf data/obj
!mkdir data/obj
!unzip /mydrive/tmp/laptops_labeled.zip -d data/obj

Archive:  /mydrive/tmp/laptops_labeled.zip
   creating: data/obj/laptops_labeled/
  inflating: data/obj/laptops_labeled/5.jpg  
  inflating: data/obj/laptops_labeled/3.txt  
  inflating: data/obj/laptops_labeled/10.jpg  
  inflating: data/obj/laptops_labeled/4.jpg  
  inflating: data/obj/laptops_labeled/7.jpg  
  inflating: data/obj/laptops_labeled/2.jpg  
  inflating: data/obj/laptops_labeled/5.txt  
  inflating: data/obj/laptops_labeled/6.jpg  
  inflating: data/obj/laptops_labeled/1.jpg  
  inflating: data/obj/laptops_labeled/12.txt  
  inflating: data/obj/laptops_labeled/11.txt  
  inflating: data/obj/laptops_labeled/1.txt  
  inflating: data/obj/laptops_labeled/11.jpg  
  inflating: data/obj/laptops_labeled/4.txt  
  inflating: data/obj/laptops_labeled/6.txt  
  inflating: data/obj/laptops_labeled/9.txt  
  inflating: data/obj/laptops_labeled/8.txt  
  inflating: data/obj/laptops_labeled/3.jpg  
  inflating: data/obj/laptops_labeled/9.jpg  
  inflating: data/obj/laptops_labeled/12

## 학습 위한 설정 파일 생성

학습에 직접 사용할 data/obj.names, data/obj.data, data/train.txt 파일을 생성한다.

<br>

data/obj.names
```
laptop
keyboard
```

<br>

data/obj.data
```
classes = 80
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/tmp/custom_object_detection
```

<br>

data/train.txt
```
data/obj/9.jpg
data/obj/12.jpg
data/obj/5.jpg
data/obj/3.jpg
data/obj/1.jpg
...
```



### data/obj.names

In [13]:
%%writefile data/obj.names
laptop
keyboard

Writing data/obj.names


In [14]:
!cat data/obj.names

laptop
keyboard

### data/obj.data

In [36]:
%%writefile data/obj.data
classes = 80
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/tmp

Overwriting data/obj.data


In [37]:
!cat data/obj.data

classes = 80
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/tmp

### data/train.txt


In [17]:
!ls -al data/obj/laptops_labeled

total 764
drwxr-xr-x 2 root root  4096 Jan  5 02:57 .
drwxr-xr-x 3 root root  4096 Jan  5 18:14 ..
-rw-rw-r-- 1 root root 88643 May  6  2021 10.jpg
-rw-rw-r-- 1 root root   151 Aug 26 20:27 10.txt
-rw-rw-r-- 1 root root 66846 May  6  2021 11.jpg
-rw-rw-r-- 1 root root    75 Aug 26 20:27 11.txt
-rw-rw-r-- 1 root root 40143 May  6  2021 12.jpg
-rw-rw-r-- 1 root root    75 Aug 26 20:27 12.txt
-rw-rw-r-- 1 root root 67386 May  6  2021 1.jpg
-rw-rw-r-- 1 root root    75 Aug 26 20:27 1.txt
-rw-rw-r-- 1 root root 58558 May  6  2021 2.jpg
-rw-rw-r-- 1 root root    75 Aug 26 20:27 2.txt
-rw-rw-r-- 1 root root  6796 May  6  2021 3.jpg
-rw-rw-r-- 1 root root    75 Aug 26 20:27 3.txt
-rw-rw-r-- 1 root root 89436 May  6  2021 4.jpg
-rw-rw-r-- 1 root root    75 Aug 26 20:27 4.txt
-rw-rw-r-- 1 root root 45128 May  6  2021 5.jpg
-rw-rw-r-- 1 root root    75 Aug 26 20:27 5.txt
-rw-rw-r-- 1 root root 38503 May  6  2021 6.jpg
-rw-rw-r-- 1 root root    37 Aug 26 20:27 6.txt
-rw-rw-r-- 1 root root 39121 Ma

In [18]:
# ls -al data/obj/laptops_labeled 밑에 jpg 파일들이 있으면
images_list = glob.glob("data/obj/laptops_labeled/*[jpg|png|jpeg]")

# ls -al data/obj 밑에 jpg 파일들이 있으면
# images_list = glob.glob("data/obj/*[jpg|png|jpeg]")

print(images_list)

# Create train.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

['data/obj/laptops_labeled/8.jpg', 'data/obj/laptops_labeled/10.jpg', 'data/obj/laptops_labeled/12.jpg', 'data/obj/laptops_labeled/2.jpg', 'data/obj/laptops_labeled/6.jpg', 'data/obj/laptops_labeled/11.jpg', 'data/obj/laptops_labeled/1.jpg', 'data/obj/laptops_labeled/7.jpg', 'data/obj/laptops_labeled/5.jpg', 'data/obj/laptops_labeled/9.jpg', 'data/obj/laptops_labeled/3.jpg', 'data/obj/laptops_labeled/4.jpg']


In [19]:
!cat data/train.txt

data/obj/laptops_labeled/8.jpg
data/obj/laptops_labeled/10.jpg
data/obj/laptops_labeled/12.jpg
data/obj/laptops_labeled/2.jpg
data/obj/laptops_labeled/6.jpg
data/obj/laptops_labeled/11.jpg
data/obj/laptops_labeled/1.jpg
data/obj/laptops_labeled/7.jpg
data/obj/laptops_labeled/5.jpg
data/obj/laptops_labeled/9.jpg
data/obj/laptops_labeled/3.jpg
data/obj/laptops_labeled/4.jpg

# 4. 사전 학습된 YOLOv3 모델 파일 다운로드

In [20]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-01-05 18:15:30--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  20.8MB/s    in 8.3s    

2022-01-05 18:15:39 (18.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# 5. 학습관련 설정 파일 수정

원 설정파일 yolov3.cfg를 yolv3_custom.cfg로 복사하고

복사한 설정파일을 수정

In [46]:
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

In [47]:
max_batches = 80 * 2000
max_batches = 100

# Edit classes & filters
!sed -i 's/batch=1/batch=64/' cfg/yolov3_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=64/' cfg/yolov3_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = $max_batches/' cfg/yolov3_custom.cfg

# 6. 학습 실행

학습된 모델은  /mydrive/tmp/yolov3_custom_last.weights 로 저장된다.

In [48]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 total_bbox = 11534, rewritten_bbox = 0.208080 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.683170), count: 2, class_loss = 276.622650, iou_loss = 1.207489, total_loss = 277.830139 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 963.384827, iou_loss = 0.000000, total_loss = 963.384827 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 3115.879639, iou_loss = 0.000000, total_loss = 3115.879639 
 total_bbox = 11536, rewritten_bbox = 0.208044 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.515646), count: 2, class_loss = 270.739227, iou_loss = 1.927307, total_loss = 272.666534 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 967.724487, iou_loss = 0.000000, total_loss = 967.724487 
v

## 7. 탐지 실행

In [49]:
!./darknet detect cfg/yolov3_custom.cfg /mydrive/tmp/yolov3_custom_last.weights data/obj/laptops_labeled/1.jpg

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
train: 27%
bird: 24%
dog: 27%
sheep: 27%
bear: 26%
zebra: 25%
handbag: 29%
tie: 30%
sports ball: 25%
baseball glove: 25%
skateboard: 31%
surfboard: 30%
orange: 25%
broccoli: 28%
hot dog: 27%
pottedplant: 30%
tvmonitor: 30%
cell phone: 28%
microwave: 26%
sink: 25%
toothbrush: 24%
bowl: 25%
chair: 44%
car: 27%
motorbike: 31%
train: 31%
truck: 43%
boat: 33%
traffic light: 37%
fire hydrant: 33%
stop sign: 30%
parking meter: 41%
bird: 42%
sheep: 28%
cow: 29%
elephant: 36%
bear: 37%
zebra: 37%
backpack: 35%
umbrella: 25%
handbag: 28%
suitcase: 33%
frisbee: 29%
snowboard: 35%
kite: 40%
skateboard: 42%
surfboard: 34%
wine glass: 41%
cup: 38%
fork: 39%
knife: 27%
bowl: 27%
banana: 29%
apple: 33%
broccoli: 35%
pizza: 44%
donut: 33%
cake: 32%
pottedplant: 26%
bed: 37%
diningtable: 26%
toilet: 42%
remote: 24%
keyboard: 27%
cell phone: 33%
microwave: 38%
toaster: 38%
book: 27%
vase: 33%
scissors: 25%
teddy bear: 41%
cow: 31%
train: 28%
bench: 24%
horse: 25%
sheep

In [ ]:
pred = plt.imread('predictions.jpg')
plt.figure(figsize=(20,20))
plt.imshow(pred)
plt.show()